In [3]:
from pprint import pprint
from elasticsearch import Elasticsearch

es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic", "So0xTl9tCAdXqx4JniGO"),
    ca_certs="http_ca.crt",
)

client_info = es.info()

print("connected to elasticsearch")
pprint(client_info.body)


connected to elasticsearch
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'Ptp3XLACR3-Wmx0PcdxoZA',
 'name': 'c899a61c3425',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2025-08-24T22:05:04.526302670Z',
             'build_flavor': 'default',
             'build_hash': '0c781091a2f57de895a73a1391ff8426c0153c8d',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '10.2.2',
             'minimum_index_compatibility_version': '8.0.0',
             'minimum_wire_compatibility_version': '8.19.0',
             'number': '9.1.3'}}


#### index documents

In [4]:
es.indices.delete(index="my_index", ignore_unavailable=True)
es.indices.create(index="my_index")

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

In [5]:
import json
from tqdm import tqdm

document_ids = []
dummy_data = json.load(open("./data/dummy_data.json"))
for doc in tqdm(dummy_data):
    response = es.index(index="my_index", document=doc)
    document_ids.append(response["_id"])


100%|██████████| 3/3 [00:00<00:00,  8.95it/s]


In [6]:
document_ids

['Y7vOUJkBtbsJOPHsxygL', 'ZLvOUJkBtbsJOPHsyCgX', 'ZbvOUJkBtbsJOPHsyChM']

In [8]:
response = es.get(index="my_index", id=document_ids[0])

In [9]:
pprint(response.body)

{'_id': 'Y7vOUJkBtbsJOPHsxygL',
 '_index': 'my_index',
 '_primary_term': 1,
 '_seq_no': 0,
 '_source': {'created_on': '2024-09-22',
             'text': 'This is the first sample document text.',
             'title': 'Sample Title 1'},
 '_version': 1,
 'found': True}


In [10]:
try:
    response = es.get(index="my_index", id="non_existing_id")
except Exception as e:
    print("Document not found")
    print(e)

Document not found
NotFoundError(404, "{'_index': 'my_index', '_id': 'non_existing_id', 'found': False}")
